# Hyperparameter Tuning using HyperDrive

In the cell below, we import all the dependencies that we need to complete the project.

In [23]:
from azureml.core import (
    Workspace,
    Experiment,
    Dataset,
    ComputeTarget,
    ScriptRunConfig
)

from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData

from azureml.train.hyperdrive import (
    BanditPolicy, 
    RandomParameterSampling,
    choice, 
    loguniform, 
    HyperDriveConfig, 
    PrimaryMetricGoal
)

## Workspace

In [19]:
subscription_id = '2c48c51c-bd47-40d4-abbe-fb8eabd19c8c'
resource_group = 'aml-quickstarts-239513'
workspace_name = 'quick-starts-ws-239513'

workspace = Workspace(subscription_id, resource_group, workspace_name)

## Experiment

In [20]:
experiment_name = 'edu_hf_hyperdrive_exp'
experiment = Experiment(workspace, experiment_name)

## Compute target

We assume a compute cluster with the given name has already been created.

In [21]:
compute_cluster_name = "edu-compute-cluster"
compute_target = workspace.compute_targets[compute_cluster_name]

## Dataset

We use the [heart failure dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data) from Kaggle.
We assume it has already been registered as an Azure ML dataset.

In [6]:
dataset_name = 'edu_heart_failure_dataset'
dataset = Dataset.get_by_name(workspace, name=dataset_name)

In [7]:
# Make a dataframe and take a look at it
patients = dataset.to_pandas_dataframe()
patients.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [26]:
train_cfg = ScriptRunConfig(
    source_directory="steps",
    script="train.py",
    compute_target=compute_target,
)

train_step = PythonScriptStep(
    name="train step",
    source_directory=train_cfg.source_directory,
    script_name=train_cfg.script,
    runconfig=train_cfg.run_config,
)

In [28]:
pipeline = Pipeline(workspace, steps=[train_step])

In [29]:
early_termination_policy = BanditPolicy(slack_factor=0.1)

param_sampling = RandomParameterSampling({
    "n_estimators": choice(5, 10, 20, 50, 100, 200),
    "min_samples_split": loguniform(-2, -1),
})

hyperdrive_run_config = HyperDriveConfig(
    pipeline=pipeline,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="AUC_weighted",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100,
)

Use of pipeline parameter in HyperDriveConfig is a preview feature. Please keep in mind that the feature can change without notice.


In [30]:
remote_run = experiment.submit(hyperdrive_run_config)

HyperDriveConfigException: HyperDriveConfigException:
	Message: Found a parameter in HyperDrive parameter space that is not defined as a pipeline parameter.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Found a parameter in HyperDrive parameter space that is not defined as a pipeline parameter."
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

